In [ ]:
!pip install konlpy

import re
import sqlite3
from konlpy.tag import Okt
from collections import Counter

# db 자료 불러오기
conn = sqlite3.connect('news.db')
c = conn.cursor()
c.execute('SELECT news_content from news_dummy2')
# c.execute('SELECT news_content from news_dummy2 WHERE pubdate LIKE "%23:55')
news_con = []
for row in c.fetchall():
  news_con.append(row)

# test용 -> 삭제하시고 news_con을 각 일자별 기사로 분류해서 넣기
news_test = news_con[5:9]

okt = Okt()
stopwords = ['코로나', '올림픽', '폐렴', '바이러스', '백신', '수급', '방역', '거리두기', '일정', '캠페인', '대해', '에서', '이고', '라고', '다고', '라기', '라며', '면서', '라면서', '로써', '로서', '으로', '에서', '어야', '부터', '한다', '이다', '였다', '였었다', '|', '/','‘', '’', ',' , '“', '”', '.', '>', '<', ')', '(']  # 제외단어 리스트
news_nouns = list()  # 기사에 사용된 모든 명사를 리스트로 담음
for news in news_test:  # news_con에서 한 줄(한 기사)씩 뽑아서 반복
  temp = ' '.join(news)  # 띄어쓰기를 넣어 문자열을 붙여줌
  temp_re = re.sub('[^가-힣]','', temp) # 가-힣을 제외한 모든 글자를 삭제 -> 이미 한글 기사만 남아있는 경우 삭제 가능 (밑에 temp_re -> temp로 수정
  encoded = okt.nouns(temp_re)  # okt를 사용하여 명사 단위로 끊음
  exist_word = [item for item in encoded if item not in stopwords]  # stopword에 있는 단어를 제외한 것을 exist_word에 넣음
  for prep_word in exist_word:  # exist_word 리스트 안의 단어를 하나씩 뽑아서 반복
    if len(prep_word) > 1:  # 뽑아낸 단어 중 2글자 이상인 것만
      news_nouns.append(prep_word)  # news_nouns 리스트에 삽입
  noun_counter = Counter(news_nouns)  # news_nouns 리스트의 문자의 빈도수를 셈

print(news_test)
print(news_nouns)
print(noun_counter)

conn.close()